In [ ]:
from yargy import rule, and_,or_, Parser
from yargy.predicates import gte, lte, dictionary,gram, is_capitalized
from yargy.interpretation import fact
from yargy.pipelines import morph_pipeline
from yargy.relations import gnc_relation

In [ ]:
gnc_nm = gnc_relation()
Name = fact(
    'Name',
    ['first','last']
)
NAME = or_(rule(
    gram("Surn").match(gnc_nm).interpretation(
        Name.last.inflected()
    ),
    gram("Name").match(gnc_nm).interpretation(
        Name.first.inflected()
    ),
    and_(
        is_capitalized(),
        gram("NOUN")
    ).optional()
),rule(
 gram("Name").match(gnc_nm).interpretation(
        Name.first.inflected()
    ),
    gram("Surn").match(gnc_nm).interpretation(
        Name.last.inflected()
    ),
    and_(
        is_capitalized(),
        gram("NOUN")
    ).optional()
)
).interpretation(Name)

In [ ]:
Date = fact(
    'Date',
    ['day','month','year']
)

BIRTH = morph_pipeline([
    "родиться",
    "появляться на свет",
    "уродиться",
    "увидеть свет",
    "год",
    "год рождения"
])
    
MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'мая': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}

MONTH_NAME = dictionary(MONTHS)


YEAR = and_(
    gte(1),
    lte(2024)
    )
MONTH = and_(
    gte(1),
    lte(12)
)
DAY = and_(
    gte(1),
    lte(31)
)
Birth_date=fact(
    'Birth',
    ['date']
)

DATE =or_( 
rule(DAY.interpretation(Date.day.custom(int)),'.',MONTH.interpretation(Date.month.custom(int)),'.',YEAR.interpretation(Date.year.custom(int))),
rule(DAY.interpretation(Date.day.custom(int)),'-',MONTH.interpretation(Date.month.custom(int)),'-',YEAR.interpretation(Date.year.custom(int))),
rule(DAY.interpretation(Date.day.custom(int)),'/',MONTH.interpretation(Date.month.custom(int)),'/',YEAR.interpretation(Date.year.custom(int))),
rule(DAY.interpretation(Date.day.custom(int)), MONTH_NAME.interpretation(Date.month.normalized().custom(MONTHS.get)), YEAR.interpretation(Date.year.custom(int))),
rule(YEAR.interpretation(Date.year.custom(int)), 'г', '.'),
rule(YEAR.interpretation(Date.year.custom(int)), 'годy')
).interpretation(Date).named('DATE')
BIRTH_DATE = or_(
    rule(
        BIRTH,
        DATE.interpretation(Birth_date.date)
         ),
    rule(
        DATE.interpretation(Birth_date.date),
        BIRTH
    )
).interpretation(Birth_date).named('BIRTH_DATE')

In [ ]:
gnc_pl = gnc_relation()
Place=fact(
    'Place',
    ['name']
)
TYPE =  morph_pipeline([
    "город",
    "страна",
    "поселок",
    "населенный пункт",
    "поселок городского типа",
    "деревня"
])
DESIGNATION = and_(
        is_capitalized(),
        gram("NOUN")
    )
PLACE = rule(
    gram("PREP"),
    TYPE.optional().match(gnc_pl),
    DESIGNATION.match(gnc_pl).interpretation(Place.name.inflected())

).interpretation(Place).named('PLACE')


In [ ]:
Final = fact(
    'Final',
    ['name','date','place']
)

S = rule(
    NAME.interpretation(Final.name),
    BIRTH_DATE.optional().interpretation(Final.date),
    gram('NOUN').optional(),
    PLACE.optional().interpretation(Final.place)
).interpretation(Final).named('INFO')
text = 'Светлана Иванова родилась 25 января 2001 года в городе Томске'
parser = Parser(S)
for match in parser.findall(text):
    display(match.fact)


In [ ]:
import gzip

from dataclasses import dataclass
from typing import Iterator

@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

texts = list(read_texts("news.txt.gz"))

In [ ]:
from tqdm import tqdm 

for text in tqdm(texts, disable=False):
    try:
        for match in parser.findall(text.text):
         print(match.fact)
    except:
       # empty
       pass